In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge

In [ ]:
df = pd.read_csv("P:\Priyank\Georgia Tech MS\Courses\Fall 2021\DVA CSE6242\Project\TransactDataClean_3110.csv")
df.head()

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Date'] = df['InvoiceDate'].dt.normalize()
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['Sales'] = df['Quantity'] * df['Price']

In [ ]:
df_prod = df.groupby(['StockCode','Date'],as_index=False).agg({'Quantity':'sum',
                                              'Sales':'sum','Customer ID':'count'})

df_prod.head()

In [ ]:
df_topProds = df_prod.groupby(['StockCode'],as_index=False)['Sales'].sum()
totalsales = df_topProds['Sales'].sum()
df_topProds.head()

In [ ]:
df_topProds['SalesContri'] = df_topProds['Sales']/totalsales
df_topProds = df_topProds.sort_values(['SalesContri'],ascending=False)
df_topProds['cum_sum'] = df_topProds['SalesContri'].cumsum()
df_topProds.head()

In [ ]:
df_topProds = df_topProds.loc[df_topProds.cum_sum<0.8]
df_topProds.shape


In [ ]:
df_salesprod = df_prod.loc[df_prod.StockCode.isin(df_topProds.StockCode),]
df_salesprod['price'] = df_salesprod['Sales']/df_salesprod['Quantity']  
df_salesprod['price'] = np.round(df_salesprod['price'],2)
df_salesprod.head()

In [ ]:
df_salesprod.replace([np.inf, -np.inf], np.nan, inplace=True)
df_salesprod = df_salesprod[df_salesprod['price'].notna()]

df_drivermodel = df_salesprod.groupby(['StockCode'],as_index=False)['Sales'].sum()
df_drivermodel['PriceBeta_LR'] = None
df_drivermodel['Accuracy_LR'] = None

df_drivermodel['PriceBeta_EN'] = None
df_drivermodel['Accuracy_EN'] = None

df_drivermodel['PriceBeta_BR'] = None
df_drivermodel['Accuracy_BR'] = None


df_drivermodel.head()

In [ ]:
df_drivermodel.head()

In [ ]:
for i in df_salesprod.StockCode.unique():
    
    X_train = df_salesprod.loc[(df_salesprod.StockCode==i),['price','Customer ID']]
    X_train = X_train.reset_index()
    X_train.drop(['index'],axis=1,inplace=True)
    
    y_train = df_salesprod.loc[(df_salesprod.StockCode==i),['Quantity']]
    y_train = y_train.reset_index()
    y_train.drop(['index'],axis=1,inplace=True)
    regr = LinearRegression(fit_intercept=True)

    regr.fit(X_train, y_train)
    df_drivermodel.loc[df_drivermodel.StockCode==i,'Accuracy_LR'] = regr.score(X_train, y_train)
    df_drivermodel.loc[df_drivermodel.StockCode==i,'PriceBeta_LR'] = regr.coef_[0][0]
    
    Enet = ElasticNet(alpha=1.0, l1_ratio=0.2)
    Enet.fit(X_train, y_train)
    #cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    #scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

    df_drivermodel.loc[df_drivermodel.StockCode==i,'Accuracy_EN'] = Enet.score(X_train, y_train)
    df_drivermodel.loc[df_drivermodel.StockCode==i,'PriceBeta_EN'] = Enet.coef_[0]
    
    
    Bayes = BayesianRidge()
    Bayes.fit(X_train, y_train)
    
    df_drivermodel.loc[df_drivermodel.StockCode==i,'Accuracy_BR'] = Bayes.score(X_train, y_train)
    df_drivermodel.loc[df_drivermodel.StockCode==i,'PriceBeta_BR'] = Bayes.coef_[0]

    

In [ ]:
df_drivermodel.head()

In [ ]:
df_drivermodel.to_csv("P:\Priyank\Georgia Tech MS\Courses\Fall 2021\DVA CSE6242\Project\SalesDriver.csv",index=False)